# 🧭 Анализ текущего уровня потребительской лояльности по результатам NPS-опросов

Исследование проводилось для крупной телекоммуникационной компании, работающей на территории СНГ.  
**Цель** — определить уровень лояльности (NPS) среди клиентов из России и построить дашборд, на основе которого можно анализировать пользовательское поведение и выделять наиболее лояльные группы.

---

### 📊 Ссылка на дашборд в Tableau:  
[https://public.tableau.com/app/profile/liliy4269/viz/project2_16743192457890/Dashboard1](https://public.tableau.com/app/profile/liliy4269/viz/project2_16743192457890/Dashboard1)

### 📄 Презентация (PDF):  
[Скачать презентацию](https://disk.yandex.ru/i/67v8XvoJ4XLngQ)


In [ ]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

<div class="alert alert-info">Подключаемся к базе данных

In [ ]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

<div class="alert alert-info">Сохраняем данные в датафрейм. Учтем сразу наличие пропусков в age age_segment, удалим эти строки, так акк их количество не большое и значительного влияние не окажет

In [ ]:
#query = """
#SELECT * FROM location;
#"""

In [ ]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'новый'
           WHEN u.lt_day > 365 THEN 'старый'
       END AS is_new,
       age,
       CASE
           WHEN u.gender_segment == 0 THEN 'мужчина'
           WHEN u.gender_segment == 1 THEN 'женщина'
       END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       SUBSTR(a_s.title, 4, 8) AS age_segment,
       SUBSTR(t_s.title, 4, 8) AS traffic_segment,
       SUBSTR(l_s.title, 4, 8) AS lifetime_segment,
       u.nps_score,
       CASE
           WHEN u.nps_score >= 9 THEN 'сторонники'
           WHEN u.nps_score == 7 OR nps_score ==8 THEN 'нейтралы'
           WHEN u.nps_score < 7 THEN 'критики'
       END AS nps_group
FROM user AS u
LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_segment AS a_s ON u.age_gr_id=a_s.age_gr_id
LEFT JOIN traffic_segment AS t_s ON u.tr_gr_id=t_s.tr_gr_id
LEFT JOIN lifetime_segment AS l_s ON u.lt_gr_id=l_s.lt_gr_id
WHERE age IS NOT NULL AND gender_segment IS NOT NULL;
"""

In [ ]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,старый,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,старый,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники
3,A004TB,4190,старый,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,сторонники
4,A004XT,1163,старый,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонники
5,A005O0,5501,старый,42.0,женщина,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,критики
6,A0061R,1236,старый,45.0,мужчина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,сторонники
7,A009KS,313,новый,35.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,сторонники
8,A00AES,3238,старый,36.0,женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,сторонники
9,A00F70,4479,старый,54.0,женщина,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,сторонники


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501152 entries, 0 to 501151
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           501152 non-null  object 
 1   lt_day            501152 non-null  int64  
 2   is_new            501152 non-null  object 
 3   age               501152 non-null  float64
 4   gender_segment    501152 non-null  object 
 5   os_name           501152 non-null  object 
 6   cpe_type_name     501152 non-null  object 
 7   country           501152 non-null  object 
 8   city              501152 non-null  object 
 9   age_segment       501152 non-null  object 
 10  traffic_segment   501152 non-null  object 
 11  lifetime_segment  501152 non-null  object 
 12  nps_score         501152 non-null  int64  
 13  nps_group         501152 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.5+ MB


In [ ]:
df.to_csv('telecomm_csi_tableau.csv', index=False)